In [1]:
using Oceananigans
using Oceananigans.TurbulenceClosures: with_tracers
using Oceananigans.BoundaryConditions: fill_halo_regions!
using Oceananigans.Fields: ZeroField, ConstantField
using Oceananigans.Models.HydrostaticFreeSurfaceModels: tracernames

using Enzyme
using LinearAlgebra

using ClimaSeaIce
using ClimaSeaIce: melting_temperature
using ClimaSeaIce.HeatBoundaryConditions: RadiativeEmission, IceWaterThermalEquilibrium

# Ocean Sea-Ice Inverse Problem

As a (very!) simplified model, suppose we have a block of ice floating on top of a body of water. The water is calm, with no current, but it has a temperature gradient which will diffuse over time. This will change the surface temperature of the water, which will in turn affect the growth/melt rate of the ice block.

We'll implement this model using two Julia packages: `Oceananigans.jl` for ocean-flavored fluid dynamics, and `ClimaSeaIce.jl` for ice thermodynamics and dynamics with salt. Both of these packages are designed for use with GPUs in mind, but for this tutorial we will only use CPUs since our problem will be fairly small. Both of these packages were written by scientists at the Climate Modeling Alliance (CliMA: https://clima.caltech.edu), and thus use very similar conventions.

To differentiate this model, we'll use `Enzyme`, a tool that performs automatic differentiation (AD) of code. `Enzyme` operates at the LLVM level (low-level virtual machine), which makes it usable for several programming languages. Here we'll use its Julia bindings, `Enzyme.jl`.

### Step 1: Set up ocean and sea ice grid

Both Oceananigans and ClimaSeaIce feature a `struct` called `grid`, which discretizes the model domain and stores some important information:
- The hardware architecture our model is running on (CPU or GPU).
- The coordinate system of our grid (we're using rectilinear coordinates).
- The spatial dimensions of our grid.
- The resolution in each dimension.
- The topology of the domain boundaries, i.e. whether boundaries are bounded or periodic.

Our ocean grid will have three dimensions labeled x, y, and z. Our ice grid will only have x and y coordinates, since our ice occupies only a single spatial layer on top of the ocean.

In [3]:
# Here we will set some important model parameters. First we specify the architecture:
arch = CPU()

# Next we set the spatial resolution of the problem. We'll keep this low so it can be run locally:
Nx = Ny = 64
Nz = 8

# Here we set the horizontal and vertical distances of the problem:
x = y = (-π, π)
z = (-0.5, 0.5)
topology = (Bounded, Bounded, Bounded)

# Here we specify the grid - the spatial discretization of our model:
grid = RectilinearGrid(arch, 
                       size=(Nx, Ny, Nz);
                       x, y, z, topology)

# And here we set up the ice grid. Note that is is flat in the z-direction since all ice is at the surface:
ice_grid = RectilinearGrid(arch,
                           size = (Nx, Ny),
                           topology = (topology[1], topology[2], Flat),
                           x = x,
                           y = y)


(u = 65×64×1 Field{Face, Center, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Nothing, east: Nothing, south: ZeroFlux, north: ZeroFlux, bottom: Nothing, top: Nothing, immersed: ZeroFlux
├── indices: (:, :, 8:8)
└── data: 71×70×1 OffsetArray(view(::Array{Float64, 3}, :, :, 11:11), -2:68, -2:67, 8:8) with eltype Float64 with indices -2:68×-2:67×8:8
    └── max=39.9518, min=-39.9518, mean=-2.44463e-17, v = 64×65×1 Field{Center, Face, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: ZeroFlux, east: ZeroFlux, south: Nothing, north: Nothing, bottom: Nothing, top: Nothing, immersed: ZeroFlux
├── indices: (:, :, 8:8)
└── data: 70×71×1 OffsetArray(view(::Array{Float64, 3}, :, :, 11:11), -2:67, -2:68, 8:8) with elt

### Step 2: Set up ocean and sea ice model

With our grids set up, we can now focus on the other aspects of the model. Our ocean model needs a diffusivity to handle tracers like temperature diffusing through the water - we'll use a vertical scalar diffusivity for this. We lable this choice `diffusion`.

(*might remove this later*) We also need a velocity field for our water body. Here we'll use a time-invariant function, showing the water flows at a consistent speed and direction over time. We name our x-direction velocity `u` and our y-direction velocity `v`. We also isolate the ocean surface velocities so they can interact with the ice.

For the ice, we need variables to represent heat and salt flux, as well as the boundary condition of the bottom with the water.

Note that all of these variables use our `grid` or `ice_grid` objects, to determine what resolution and architecture they are represented on.

In [4]:
# Then we set a maximal diffusivity and diffusion type:
const maximum_diffusivity = 100
diffusion = VerticalScalarDiffusivity(κ=0.1)

# For this problem we'll have a constant-values velocity field - the water will flow at a
# constant speed and direction over time. We have to assign the proper velocity values to
# every point in our grid:
u = XFaceField(grid)
v = YFaceField(grid)

U = 40
u₀(x, y, z) = - U * cos(x + π/4) * sin(y)
v₀(x, y, z) = + U * 0.5

set!(u, u₀)
set!(v, v₀)
fill_halo_regions!(u)
fill_halo_regions!(v)

ice_ocean_heat_flux      = Field{Center, Center, Nothing}(ice_grid)
top_ocean_heat_flux = Qᵀ = Field{Center, Center, Nothing}(ice_grid)
top_salt_flux       = Qˢ = Field{Center, Center, Nothing}(ice_grid)

bottom_bc = IceWaterThermalEquilibrium(ConstantField(30)) #ocean_surface_salinity)

ocean_surface_velocities = (u = view(u, :, :, Nz), #interior(u, :, :, Nz),
                            v = view(v, :, :, Nz), #interior(v, :, :, Nz),    
                            w = ZeroField())

(u = 65×64×1 Field{Face, Center, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: Nothing, east: Nothing, south: ZeroFlux, north: ZeroFlux, bottom: Nothing, top: Nothing, immersed: ZeroFlux
├── indices: (:, :, 8:8)
└── data: 71×70×1 OffsetArray(view(::Array{Float64, 3}, :, :, 11:11), -2:68, -2:67, 8:8) with eltype Float64 with indices -2:68×-2:67×8:8
    └── max=39.9518, min=-39.9518, mean=-2.44463e-17, v = 64×65×1 Field{Center, Face, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: ZeroFlux, east: ZeroFlux, south: Nothing, north: Nothing, bottom: Nothing, top: Nothing, immersed: ZeroFlux
├── indices: (:, :, 8:8)
└── data: 70×71×1 OffsetArray(view(::Array{Float64, 3}, :, :, 11:11), -2:67, -2:68, 8:8) with elt

#### With our required variables initialized, we can now create our ocean and ice models!

Our ocean model uses the hydrostatic approximation - hence we construct a `HydrostaticFreeSurfaceModel`. In addition to supplying the diffusivity and velocity fields above, we also add a tracer `T` to represent our temperature, and the WENO advection scheme for our temperature.

Our ice model is a simple slab model with only one layer.In addition to the fluxes above, we also supply a starting salinity, internal heat flux, and top heat flux and boundary condition.

In [10]:
model = HydrostaticFreeSurfaceModel(; grid,
                                    tracer_advection = WENO(),
                                    tracers = :T,
                                    buoyancy = nothing,
                                    velocities = PrescribedVelocityFields(; u, v),
                                    closure = diffusion)

ice_model = SlabSeaIceModel(ice_grid;
                            velocities = ocean_surface_velocities,
                            advection = nothing, #WENO(),
                            ice_consolidation_thickness = 0.05,
                            ice_salinity = 4,
                            internal_heat_flux = ConductiveFlux(conductivity=2),
                            #top_heat_flux = ConstantField(-100), # W m⁻²
                            top_heat_flux = ConstantField(0), # W m⁻²
                            top_heat_boundary_condition = PrescribedTemperature(0),
                            bottom_heat_boundary_condition = bottom_bc,
                            bottom_heat_flux = ice_ocean_heat_flux)

SlabSeaIceModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 64×64×1 RectilinearGrid{Float64, Bounded, Bounded, Flat} on CPU with 3×3×0 halo
├── top_surface_temperature: ConstantField(0)
├── minimium_ice_thickness: ConstantField(0.05)
└── external_heat_fluxes: 
    ├── top: ConstantField(0)
    └── bottom: 64×64×1 Field{Center, Center, Nothing} reduced over dims = (3,) on RectilinearGrid on CPU

### Step 3: Obtain the "real" data values

Now we have a functioning ocean and sea ice model. Suppose we have the final temperature distribution and ice thickness, and wish to use these to reconstruct the initial ocean temperature and ice thickness (*this may change later*). In a real project, we might have measured temperature and thickness data to use for our inverse problem. For this tutorial, we'll instead generate "real" data values using a function called `ice_ocean_data`. For this function we'll run our ocean and ice models forward a number of time steps to produce our data values. The function `ice_ocean_data` produces both the initial temperature distribution of the water (what we aim to invert for) as well as the final temperature distribution (what we are given for the inverse problem).

In [23]:
# TODO: add sea ice component

# Sets the initial diffusivity:
function set_diffusivity!(model, diffusivity)
    closure = VerticalScalarDiffusivity(; κ=diffusivity)
    names = tuple(:T) # tracernames(model.tracers)
    closure = with_tracers(names, closure)
    model.closure = closure
    return nothing
end

# This produces an initial data distribution:
function set_initial_data!(model)
    amplitude   = Ref(1)
    width       = 0.1
    Tᵢ(x, y, z) = amplitude[] * exp(-z^2 / (2width^2))

    set!(model, T=Tᵢ)

    return nothing
end

# Generates the "real" data from a stable diffusion run:
function ice_ocean_data(model, ice_model, diffusivity, n_max)
    
    set_diffusivity!(model, diffusivity)
    set_initial_data!(model)
    
    # Do time-stepping
    Nx, Ny, Nz = size(model.grid)
    κ_max = maximum_diffusivity
    Δz = 2π / Nz
    Δt = 1e-1 * Δz^2 / κ_max

    model.clock.time = 0
    model.clock.iteration = 0

    T₀ = deepcopy(model.tracers.T)

    for n = 1:n_max
        time_step!(model, Δt; euler=true)
        # TODO: supply ocean surface temperature to sea ice model
        time_step!(ice_model, Δt)
    end

    # Compute scalar metric
    Tₙ = deepcopy(model.tracers.T)

    return T₀, Tₙ
end

stable_diffusion_data (generic function with 1 method)

### Step 4: Run the forward model

With our model structs and intial data, we can now run the actual ice-ocean model for an inverse problem. Given some initial guess for our temperature distribution, we will run the model forward the set number of time steps and compare our computed final temperature with the "true" temperature from `ice_ocean_data`.

In [24]:
function set_initial_condition!(model, Tᵢ)

    # This has a "width" of 0.1
    #Tᵢ(x, y, z) = amplitude[] * exp(-z^2 / 0.02 - (x^2 + y^2) / 0.05)
    set!(model, T=Tᵢ)

    return nothing
end

# cᵢ is the proposed initial condition, cₙ is the actual data collected
# at the final time step:
function ice_ocean_model(model, ice_model, diffusivity, n_max, Tᵢ, Tₙ)
    set_diffusivity!(model, diffusivity)
    set_initial_condition!(model, Tᵢ)
    
    # Run the forward model:
    Nx, Ny, Nz = size(model.grid)
    κ_max = maximum_diffusivity
    Δz = 2π / Nz
    Δt = 1e-1 * Δz^2 / κ_max

    model.clock.time = 0
    model.clock.iteration = 0

    for n = 1:n_max
        time_step!(model, Δt; euler=true)
        time_step!(ice_model, Δt)
    end

    T = model.tracers.T
    # Compute the misfit of our forward model run with the true data cₙ:
    J = 0.0
    for i = 1:Nx, j = 1:Ny, k = 1:Nz
        J += (T[i, j, k] - Tₙ[i, j, k])^2
    end

    return J::Float64
end

advection_diffusion_model! (generic function with 1 method)

### Step 5: Set up the inverse problem parameters

In [25]:
κ = 1
n_max  = 80
T₀, Tₙ = stable_diffusion_data(model, ice_model, κ, n_max)

@show T₀
@show Tₙ

Tᵢ = zeros(size(model.tracers.T))

learning_rate = 0.01
max_steps = 10
δ = 0.01

T₀ = 64×64×8 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: ZeroFlux, east: ZeroFlux, south: ZeroFlux, north: ZeroFlux, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 70×70×14 OffsetArray(::Array{Float64, 3}, -2:67, -2:67, -2:11) with eltype Float64 with indices -2:67×-2:67×-2:11
    └── max=0.822578, min=6.97696e-5, mean=0.250661
Tₙ = 64×64×8 Field{Center, Center, Center} on RectilinearGrid on CPU
├── grid: 64×64×8 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×3 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: ZeroFlux, east: ZeroFlux, south: ZeroFlux, north: ZeroFlux, bottom: ZeroFlux, top: ZeroFlux, immersed: ZeroFlux
└── data: 70×70×14 OffsetArray(::Array{Float64, 3}, -2:67, -2:67, -2:11) with eltype Float64 with indices -2:67×-2:67×-2:11
    └── max=1.98294, min=0

0.01

### Step 6: Run the inverse problem

In [35]:
# Update our guess of the initial tracer distribution, cᵢ:
for i = 1:max_steps
    dmodel = Enzyme.make_zero(model)
    dTᵢ = Enzyme.make_zero(Tᵢ)
    dTₙ = Enzyme.make_zero(Tₙ)
    set_diffusivity!(dmodel, 0)

    # Temporarily withheld until Oceananigans + enzyme bugs are fixed
    
    # Since we are only interested in duplicated variable cᵢ for this run,
    # we do not use dJ here:
    #=
    dJ = autodiff(Enzyme.Reverse,
                    advection_diffusion_model!,
                    Duplicated(model, dmodel),
                    Const(κ),
                    Const(n_max),
                    Duplicated(cᵢ, dcᵢ),
                    Duplicated(cₙ, dcₙ))
    
    @show i
    @show norm(dcᵢ)
    global cᵢ .= cᵢ .- (dcᵢ .* learning_rate)
    @show (norm(cᵢ - c₀) / norm(c₀))

    # Stop gradient descent if dcᵢ is sufficiently small:
    if norm(dcᵢ) < δ
        break
    end
    =#
end